In [1]:
import os.path
import re
import time
import numpy as np
import tensorflow as tf
import cifar10

# total loss function

In [2]:
def tower_loss(scope):
    images, labels = cifar10.distorted_inputs()
    # call cnn from inference
    logits = cifar10.inference(images)
    _ = cifar10.loss(logits, labels)
    losses = tf.get_collection('losses', scope)
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

# avg gradients function

In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expanded_g = tf.expand_dims(g, 0)
            grads.append(expanded_g)
            
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)
        
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

# train function

In [4]:
def train():
    with tf.Graph().as_default(), tf.device('/cpu:0'):
        global_step = tf.get_variable('global_step', [],
                                     initializer=tf.constant_initializer(0), trainable=False)
        
        num_batches_per_epoch = cifar10.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / batch_size
        decay_steps = int(num_batches_per_epoch * cifar10.NUM_EPOCHS_PER_DECAY)
        
        lr = tf.train.exponential_decay(cifar10.INITIAL_LEARNING_RATE,
                                       global_step, decay_steps, cifar10.LEARNING_RATE_DECAY_FACTOR, staircase=True)
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        # gpu gradient distributed
        tower_grads = []
        for i in range(num_gpus):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % (cifar10.TOWER_NAME, i)) as scope:
                    loss = tower_loss(scope)
                    tf.get_variable_scope().reuse_variables()
                    # single gpu gradient
                    grads = opt.compute_gradients(loss)
                    tower_grads.append(grads)
                    
        # update model gradients
        grads = average_gradients(tower_grads)
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
        
        # session and allow_soft_placement
        saver = tf.train.Saver(tf.all_variables())
        init = tf.global_variables_initializer()
        sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
        sess.run(init)
        tf.train.start_queue_runners(sess=sess)
        
        # batch loss and time
        for step in range(max_steps):
            start_time = time.time()
            _, loss_value = sess.run([apply_gradient_op, loss])
            duration = time.time() - start_time
            
            if step % 10 == 0:
                num_examples_per_step = batch_size * num_gpus
                examples_per_sec = num_examples_per_step / duration
                sec_per_batch = duration / num_gpus
                
                format_str = ('step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
                print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))
                
            if step % 1000 == 0 or (step + 1) == max_steps:
                saver.save(sess, 'cifar10/train_model.ckpt', global_step=step)

# train

In [5]:
batch_size = 128
max_steps = 10000
num_gpus = 2

cifar10.maybe_download_and_extract()
train()

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Instructions for updating:
Please use tf.global_variables instead.
step 0, loss = 4.68 (13.6 examples/sec; 9.426 sec/batch)
step 10, loss = 4.61 (1706.9 examples/sec; 0.075 sec/batch)
step 20, loss = 4.55 (1820.6 examples/sec; 0.070 sec/batch)
step 30, loss = 4.41 (1801.0 examples/sec; 0.071 sec/batch)
step 40, loss = 4.30 (1924.6 examples/sec; 0.067 sec/batch)
step 50, loss = 4.26 (1814.4 examples/sec; 0.071 sec/batch)
step 60, loss = 4.28 (1877.9 examples/sec; 0.068 sec/batch)
step 70, loss = 4.13 (1742.5 examples/sec; 0.073 sec/batch)
step 80, loss = 4.19 (1812.0 examples/sec; 0.071 sec/batch)
step 90, loss = 4.13 (1875.4 examples/sec; 0.068 sec/batch)
step 100, loss = 4.32 (1808.7 examples/sec; 0.071 sec/batch)
step 110, loss = 4.22 (1816.3 examples/sec; 0.070 sec/batch)
step 120, loss = 4.00 (188

step 1280, loss = 2.21 (1778.2 examples/sec; 0.072 sec/batch)
step 1290, loss = 2.08 (1868.0 examples/sec; 0.069 sec/batch)
step 1300, loss = 2.02 (1775.9 examples/sec; 0.072 sec/batch)
step 1310, loss = 2.09 (1841.0 examples/sec; 0.070 sec/batch)
step 1320, loss = 1.87 (1861.1 examples/sec; 0.069 sec/batch)
step 1330, loss = 1.96 (1845.0 examples/sec; 0.069 sec/batch)
step 1340, loss = 2.02 (1974.6 examples/sec; 0.065 sec/batch)
step 1350, loss = 1.95 (1930.1 examples/sec; 0.066 sec/batch)
step 1360, loss = 2.04 (1894.9 examples/sec; 0.068 sec/batch)
step 1370, loss = 1.78 (1902.7 examples/sec; 0.067 sec/batch)
step 1380, loss = 1.86 (1890.3 examples/sec; 0.068 sec/batch)
step 1390, loss = 2.02 (1760.4 examples/sec; 0.073 sec/batch)
step 1400, loss = 1.83 (1766.5 examples/sec; 0.072 sec/batch)
step 1410, loss = 1.95 (1812.6 examples/sec; 0.071 sec/batch)
step 1420, loss = 1.92 (1851.5 examples/sec; 0.069 sec/batch)
step 1430, loss = 1.86 (1887.0 examples/sec; 0.068 sec/batch)
step 144

step 2610, loss = 1.33 (1901.1 examples/sec; 0.067 sec/batch)
step 2620, loss = 1.42 (1848.0 examples/sec; 0.069 sec/batch)
step 2630, loss = 1.09 (1792.0 examples/sec; 0.071 sec/batch)
step 2640, loss = 1.41 (1781.9 examples/sec; 0.072 sec/batch)
step 2650, loss = 1.17 (1832.6 examples/sec; 0.070 sec/batch)
step 2660, loss = 1.35 (1826.6 examples/sec; 0.070 sec/batch)
step 2670, loss = 1.26 (1835.6 examples/sec; 0.070 sec/batch)
step 2680, loss = 1.35 (1812.1 examples/sec; 0.071 sec/batch)
step 2690, loss = 1.36 (1795.1 examples/sec; 0.071 sec/batch)
step 2700, loss = 1.48 (1852.6 examples/sec; 0.069 sec/batch)
step 2710, loss = 1.50 (1906.9 examples/sec; 0.067 sec/batch)
step 2720, loss = 1.22 (1888.6 examples/sec; 0.068 sec/batch)
step 2730, loss = 1.40 (1895.2 examples/sec; 0.068 sec/batch)
step 2740, loss = 1.14 (1888.6 examples/sec; 0.068 sec/batch)
step 2750, loss = 1.46 (1784.5 examples/sec; 0.072 sec/batch)
step 2760, loss = 1.20 (1822.9 examples/sec; 0.070 sec/batch)
step 277

step 3940, loss = 0.96 (1836.4 examples/sec; 0.070 sec/batch)
step 3950, loss = 0.93 (1861.0 examples/sec; 0.069 sec/batch)
step 3960, loss = 1.01 (1979.7 examples/sec; 0.065 sec/batch)
step 3970, loss = 1.06 (1831.5 examples/sec; 0.070 sec/batch)
step 3980, loss = 0.93 (1831.1 examples/sec; 0.070 sec/batch)
step 3990, loss = 0.99 (1814.0 examples/sec; 0.071 sec/batch)
step 4000, loss = 0.99 (1813.0 examples/sec; 0.071 sec/batch)
step 4010, loss = 0.89 (1812.9 examples/sec; 0.071 sec/batch)
step 4020, loss = 0.93 (1809.4 examples/sec; 0.071 sec/batch)
step 4030, loss = 0.94 (1795.8 examples/sec; 0.071 sec/batch)
step 4040, loss = 0.88 (1835.9 examples/sec; 0.070 sec/batch)
step 4050, loss = 0.91 (1824.8 examples/sec; 0.070 sec/batch)
step 4060, loss = 1.13 (1843.1 examples/sec; 0.069 sec/batch)
step 4070, loss = 0.81 (1904.4 examples/sec; 0.067 sec/batch)
step 4080, loss = 1.12 (1762.8 examples/sec; 0.073 sec/batch)
step 4090, loss = 0.84 (1868.6 examples/sec; 0.068 sec/batch)
step 410

step 5270, loss = 0.80 (1842.7 examples/sec; 0.069 sec/batch)
step 5280, loss = 0.87 (1824.5 examples/sec; 0.070 sec/batch)
step 5290, loss = 0.80 (1840.8 examples/sec; 0.070 sec/batch)
step 5300, loss = 0.87 (1773.5 examples/sec; 0.072 sec/batch)
step 5310, loss = 0.93 (1997.9 examples/sec; 0.064 sec/batch)
step 5320, loss = 0.80 (1810.3 examples/sec; 0.071 sec/batch)
step 5330, loss = 1.04 (1785.7 examples/sec; 0.072 sec/batch)
step 5340, loss = 0.98 (1878.0 examples/sec; 0.068 sec/batch)
step 5350, loss = 1.02 (1906.1 examples/sec; 0.067 sec/batch)
step 5360, loss = 0.93 (1790.4 examples/sec; 0.071 sec/batch)
step 5370, loss = 0.90 (1847.4 examples/sec; 0.069 sec/batch)
step 5380, loss = 0.85 (1762.5 examples/sec; 0.073 sec/batch)
step 5390, loss = 1.02 (1859.4 examples/sec; 0.069 sec/batch)
step 5400, loss = 0.95 (1955.8 examples/sec; 0.065 sec/batch)
step 5410, loss = 0.91 (1819.3 examples/sec; 0.070 sec/batch)
step 5420, loss = 0.76 (1921.8 examples/sec; 0.067 sec/batch)
step 543

step 6600, loss = 0.72 (1864.4 examples/sec; 0.069 sec/batch)
step 6610, loss = 0.86 (1785.7 examples/sec; 0.072 sec/batch)
step 6620, loss = 0.80 (1832.1 examples/sec; 0.070 sec/batch)
step 6630, loss = 0.77 (1885.9 examples/sec; 0.068 sec/batch)
step 6640, loss = 0.75 (1871.4 examples/sec; 0.068 sec/batch)
step 6650, loss = 0.83 (1868.1 examples/sec; 0.069 sec/batch)
step 6660, loss = 0.72 (1868.4 examples/sec; 0.069 sec/batch)
step 6670, loss = 0.87 (1778.8 examples/sec; 0.072 sec/batch)
step 6680, loss = 0.83 (1829.5 examples/sec; 0.070 sec/batch)
step 6690, loss = 0.80 (1740.4 examples/sec; 0.074 sec/batch)
step 6700, loss = 0.68 (1828.9 examples/sec; 0.070 sec/batch)
step 6710, loss = 0.87 (1801.8 examples/sec; 0.071 sec/batch)
step 6720, loss = 0.88 (1891.4 examples/sec; 0.068 sec/batch)
step 6730, loss = 0.87 (1872.7 examples/sec; 0.068 sec/batch)
step 6740, loss = 0.88 (1855.7 examples/sec; 0.069 sec/batch)
step 6750, loss = 0.70 (1826.5 examples/sec; 0.070 sec/batch)
step 676

step 7930, loss = 0.83 (1809.2 examples/sec; 0.071 sec/batch)
step 7940, loss = 0.76 (1820.6 examples/sec; 0.070 sec/batch)
step 7950, loss = 0.74 (1910.7 examples/sec; 0.067 sec/batch)
step 7960, loss = 0.81 (1828.5 examples/sec; 0.070 sec/batch)
step 7970, loss = 0.83 (1785.9 examples/sec; 0.072 sec/batch)
step 7980, loss = 0.67 (1822.5 examples/sec; 0.070 sec/batch)
step 7990, loss = 0.72 (1848.1 examples/sec; 0.069 sec/batch)
step 8000, loss = 0.75 (1864.9 examples/sec; 0.069 sec/batch)
step 8010, loss = 0.71 (1750.8 examples/sec; 0.073 sec/batch)
step 8020, loss = 0.94 (1845.1 examples/sec; 0.069 sec/batch)
step 8030, loss = 0.85 (1846.8 examples/sec; 0.069 sec/batch)
step 8040, loss = 0.70 (1954.9 examples/sec; 0.065 sec/batch)
step 8050, loss = 0.64 (1849.2 examples/sec; 0.069 sec/batch)
step 8060, loss = 0.63 (1820.3 examples/sec; 0.070 sec/batch)
step 8070, loss = 0.87 (1853.4 examples/sec; 0.069 sec/batch)
step 8080, loss = 0.74 (1935.0 examples/sec; 0.066 sec/batch)
step 809

step 9260, loss = 0.83 (1862.7 examples/sec; 0.069 sec/batch)
step 9270, loss = 0.80 (1809.0 examples/sec; 0.071 sec/batch)
step 9280, loss = 0.71 (1787.0 examples/sec; 0.072 sec/batch)
step 9290, loss = 0.85 (1825.6 examples/sec; 0.070 sec/batch)
step 9300, loss = 0.81 (1958.8 examples/sec; 0.065 sec/batch)
step 9310, loss = 0.72 (1773.5 examples/sec; 0.072 sec/batch)
step 9320, loss = 0.96 (1716.6 examples/sec; 0.075 sec/batch)
step 9330, loss = 0.92 (1828.0 examples/sec; 0.070 sec/batch)
step 9340, loss = 0.79 (1884.0 examples/sec; 0.068 sec/batch)
step 9350, loss = 0.65 (1808.4 examples/sec; 0.071 sec/batch)
step 9360, loss = 0.80 (1842.8 examples/sec; 0.069 sec/batch)
step 9370, loss = 0.73 (1857.5 examples/sec; 0.069 sec/batch)
step 9380, loss = 0.72 (1849.8 examples/sec; 0.069 sec/batch)
step 9390, loss = 0.86 (1902.0 examples/sec; 0.067 sec/batch)
step 9400, loss = 0.72 (1904.7 examples/sec; 0.067 sec/batch)
step 9410, loss = 0.79 (1844.8 examples/sec; 0.069 sec/batch)
step 942